In [1]:
import torch 
from modules.crf_model import CRFModel
import pytorch_lightning as pl

from pathlib import Path
torch.set_float32_matmul_precision('medium')

torch.manual_seed(10)

In [2]:
from modules.ner_tasks import RUSSIAN
from modules.ner_data import get_dataloaders


ner_task = RUSSIAN['WikiNeural']

In [3]:
train_dataloader, val_dataloader, test_dataloader = get_dataloaders(
    ner_task,
    batch_size=128,
    num_workers=24,
    pin_memory=True
)

In [4]:
from tqdm.notebook import tqdm

In [5]:
from modules.vectorizer import BidirectionalLM

model_params = {
    'num_layers': 2, 
    'hidden_size': 2048,
    'sru_attn_proj_size': 1024,
    'attention_last_n_layers': 0
}

vectorizer = BidirectionalLM(
    vocab_size=None, 
    model_params=model_params, 
    learning_rate=0.001, 
    cutoffs=[20_000, 50_000],
    elmo_options_file='../Elmo-taiga/options.json', 
    elmo_weight_file='../Elmo-taiga/model.hdf5',
)

vectorizer.load_state_dict(torch.load(
    'elmo_lstm_2lrs_checkpoints/epoch=0-step=9162.ckpt'
)['state_dict'], strict=False)
vectorizer.freeze()

In [ ]:

vectorizer.emb_size = 1024
ner_model = CRFModel(
    ner_labels=ner_task['labels_vocab'], 
    vectorizer=vectorizer, 
    weight_decay=0, 
    learning_rate=1e-3
)

trainer = pl.Trainer(
    accelerator='gpu', 
    gradient_clip_val=10.5, 
    max_epochs=5, 
)

trainer.fit(ner_model, train_dataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/media/fast_data/anaconda/lib/python3.10/site-packages/pytorch_lightning/trainer/configuration_validator.py:72: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type            | Params
-----------------------------------------------
0 | vectorizer | BidirectionalLM | 129 M 
1 | lstm       | LSTM            | 1.4 M 
2 | relu       | ReLU            | 0     
3 | line       | Linear          | 2.2 K 
4 | crf        | CRF             | 99    
-----------------------------------------------
1.4 M     Trainable params
129 M     Non-trainable params
130 M     Total params
522.814   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

In [ ]:
trainer.validate(ner_model, test_dataloader)